# Exercise 9: Recommender Systems

**Libaries needed: scikit-surprise, pandas, sklearn, numpy. 
To install `scikit-surprise`:**
```
conda install -c conda-forge scikit-surprise
```

In this exercise, we will be proceeding in two stages. The first stage is where we get into the details of how to build our own recommender system to recommend movies to users. In the second stage, we will be an existing library, specialized for recommender systems, which provides more powerful options. We will be testing it on the task of recommending jokes to users. 

Let's make sure we have all the requirements ready. In this exercise, you should be filling the empty code sections, marked as `TODO:`

In [1]:
import surprise
import numpy as np
import pandas as pd
import sklearn

### Task 1: Exploring the MovieLens dataset

In this part, we'll be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/). This dataset is based on [movielens.org](https://movielens.org/), a site where users can get movie recommendations.

Our first step is to load the relevant file of the dataset, which you can find in the file `u.data` (on the path `data/ml-100k/u.data`).


In [2]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('data/ml-100k/u.data', sep='\t', names=header)

In [3]:
df[:15]

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


Let's first check the number of users and movies in the dataset to get an idea of the scale we're dealing with.

In [8]:
# TODO: get the number of users and itens
n_users = len(np.unique((df['user_id']).get_values()))
n_items = len(np.unique((df['item_id']).get_values()))
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))

Number of users = 943 | Number of movies = 1682


We can also get an overall view of the dataset as below. Notice how the ratings range from a minimum of 1 to a maximum of 5.

In [9]:
df.describe()

,user_id,item_id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


Now that the data is loaded, we proceed to splitting it into a training set and a testing set.


In [10]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.25, random_state= 42)

Next, let's create the user-item matrices, one for training and another for testing. Each matrix should be a 2D numpy array, with each row corresponding to a user and each column to a movie. A non-zero cell in the matrix is the rating given by the user to the movie (zeros are for the case of no corresponding rating).

**Notice that the user ids and item ids start from 1, so the index (0,0) in your matrix should correspond to `user_id` of 1 and `item_id` of 1.**

In [19]:
# TODO fill the code to produce a data matrix
def create_data_matrix(data,n_users,n_items):
    """
        This function should return a numpy matrix with a shape (n_users, n_items). 
        Each entry is the rating given by the user to the item
    """
    data_matrix = np.zeros((n_users,n_items))
    matrix = data.values
    for i in range(0,len(matrix)):
        user_id = matrix[i][0]
        movie_id = matrix[i][1]
        rating = matrix[i][2]
        data_matrix[user_id-1][movie_id-1] = rating
    
    
    return data_matrix

train_data_matrix= create_data_matrix(train_data, n_users, n_items)
test_data_matrix= create_data_matrix(test_data, n_users, n_items)

We can check how our matrices look like at this point. 

In [20]:
print('train_data_matrix')
print(train_data_matrix)
print('test_data_matrix')
print(test_data_matrix)

train_data_matrix
[[0. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
test_data_matrix
[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Task 2: Item-based Collaborative Filtering

Now that we've prepared our data, the next mission we have is to create a recommender system following the paradigm of Item-based Collaborative Filtering. In this case, this is translated into "Users who liked this item (movie) also liked …". 



In order to make predictions, we will apply following formula, where 
$N_I(a)$ is the set of neighbors of item $a$, and $b$ is an item rated by user $x$.


\begin{equation}
{r}_{x}(a) =  \frac{\sum\limits_{b \in N_{I}(a)} sim(a, b) r_{x}(b)}{\sum\limits_{b \in N_{I}(a)}|sim(a, b)|}
\end{equation}

As a building block, we'll first write the code for the similarity $sim(a,b)$ metric between each two item vectors in our training matrix. In this case, we will use the cosine similarity metric. The output should be an `n_items` by `n_items` symmetric 2D numpy matrix with the similarity between each couple of items.

**Note**: In this exercise, there are always two ways of achieving the same goal: a slow one via `for` loops and another by benefiting from numpy's speed in matrix operations. Feel free to improve your starting solution to a faster one.

In [59]:
# TODO fill the code to compute the similarity matrix
from sklearn.metrics.pairwise import pairwise_distances

transpose = train_data_matrix.T
item_similarity = pairwise_distances(transpose)

# check how the matrix looks like
print(item_similarity.shape)

(1682, 1682)


Next, we'll use the similarity matrix in the above formula to obtain the predicted ranking for each item `a`.

In [60]:
# TODO: Fill the code for predicting the ratings. 
# The output is a numpy matrix with the dimensions ((n_users,n_items)) and with the corresponding ranking at each cell.
def item_based_predict(ratings, similarity):
    pred = np.dot(ratings,similarity)
    abso = np.sum(np.abs(similarity),axis=1)
    return pred/abso

item_prediction = item_based_predict(train_data_matrix, item_similarity)
print(item_prediction)


[[0.44016055 0.54125864 0.59196504 ... 0.92275985 0.91115111 0.93394596]
 [0.10031651 0.14207273 0.15232148 ... 0.24238889 0.23972821 0.24565403]
 [0.07379637 0.08693234 0.09058243 ... 0.12741225 0.12656886 0.12938732]
 ...
 [0.03734345 0.05778127 0.06242303 ... 0.10373337 0.10240622 0.1050807 ]
 [0.1578904  0.1976772  0.21865603 ... 0.33447082 0.33044667 0.33875886]
 [0.25853288 0.31424303 0.35904595 ... 0.5654731  0.55798925 0.57229893]]


### Task 3: User-based Collaborative Filtering

The next mission we have is to create a recommender system following the paradigm of User-based Collaborative Filtering. In this case, this is translated into "Users who are similar to you also liked…". 

In order to make predictions, we will apply following formula, where $N_U(x)$ is the set of neighbors of user x and $a$ is an item not rated by x.


\begin{equation}
{r}_{x}(a) = \bar{r}_{x} + \frac{\sum\limits_{y \in N_{U}(x)} sim(x, y) (r_{y}(a) - \bar{r}_{y})}{\sum\limits_{y \in N_{U}(x)}|sim(x, y)|}
\end{equation}

Similar to above, we will first compute the distances between the users in our training matrix, using cosine similarity. The output should be an `n_users` by `n_users` symmetric 2D numpy matrix with the similarity between each couple of users.

In [62]:
# TODO fill the code to compute the similarity matrix
user_similarity = pairwise_distances(train_data_matrix)

# print the shape as a sanity check
print(user_similarity.shape)

# check how the matrix looks like
print(user_similarity)

(943, 943)
[[ 0.         57.33236433 57.75811631 ... 56.06246516 61.91930232
  57.2974694 ]
 [57.33236433  0.         30.87069808 ... 29.93325909 39.50949253
  46.73328578]
 [57.75811631 30.87069808  0.         ... 25.70992026 37.20215048
  45.4202598 ]
 ...
 [56.06246516 29.93325909 25.70992026 ...  0.         36.7559519
  43.28972164]
 [61.91930232 39.50949253 37.20215048 ... 36.7559519   0.
  49.92995093]
 [57.2974694  46.73328578 45.4202598  ... 43.28972164 49.92995093
   0.        ]]


In [64]:
# TODO: Fill the code for predicting the ratings. 
def user_based_predict(ratings, similarity):
    normalize = ratings - ((np.mean(similarity,axis=0)).reshape(-1,1))
    pred = np.dot(similarity,ratings)
    abso = np.sum(np.abs(similarity),axis=1)
    return pred

user_prediction = user_based_predict(train_data_matrix, user_similarity)
print(user_prediction)


[[7.30951915e+04 1.89416097e+04 1.20616268e+04 ... 1.24851912e+02
  1.79799889e+02 0.00000000e+00]
 [5.24020414e+04 1.61968861e+04 9.85577792e+03 ... 7.31026675e+01
  1.66303939e+02 0.00000000e+00]
 [5.09773821e+04 1.58142117e+04 9.70375730e+03 ... 5.97662112e+01
  1.64918162e+02 0.00000000e+00]
 ...
 [4.80082080e+04 1.53514683e+04 9.32954216e+03 ... 7.18888030e+01
  1.60521027e+02 0.00000000e+00]
 [5.88730160e+04 1.69758731e+04 1.07861534e+04 ... 8.74299720e+01
  1.69493363e+02 0.00000000e+00]
 [6.13251277e+04 1.62300361e+04 1.06758981e+04 ... 1.04880885e+02
  1.61972220e+02 0.00000000e+00]]


### Task 4: Evaluating Our Recommenders

We will be evaluating our recommenders using Root Mean Squared Error (RMSE). In the formula below, $r_i$ is the true rating and $\hat{r_i}$ is the predicted one.

\begin{equation}
\mathit{RMSE} =\sqrt{\frac{1}{N} \sum_i (r_i -\hat{r_i})^2}
\end{equation}

In [68]:
# TODO: add the code for computing RMSE for user and item based methods
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    sqr_diff = np.power(ground_truth - prediction,2)
    total = np.sum(sqr_diff)/prediction.shape[1]
    return sqrt(total)

print ('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print ('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 416521.92557338317
Item-based CF RMSE: 17.247072003019458


### Task 5: Introducing Surprise

In this part, we will move to using [Surprise](http://surpriselib.com/), a full-fledged python library, specialized for recommender systems. The goal is to get exposed to such more powerful libraries that can automate a lot of the manual work we had to do above.

For a change, we will be using the [Jester](http://eigentaste.berkeley.edu/dataset/) dataset, obtained from the [Jester Online Joke Recommender System](http://eigentaste.berkeley.edu/index.html). It has over 1.7 million continuous ratings (-10.00 to +10.00) of 150 jokes from 59,132 users: collected between November 2006 - May 2009. Our first step will be to download this dataset. Fortunately, `Surprise` has a built-in loader for the Jester dataset. Make sure you confirm that you want to download the dataset when prompted to do so.

In [71]:
from surprise import Dataset


# Load the Jester dataset (download it if needed),
data = Dataset.load_builtin('jester')
# split the data into 2 folds for cross-validation.
data.split(n_folds=2)



Next, we will need to train the k-Nearest Neighbors algorithm within Surprise on the Jester dataset (Check the [documentation](http://surprise.readthedocs.io/en/stable/) for `SVD`). For evaluation, Jester allows multiple metrics. You will need to use the `RMSE` and the `MAE` in this case. The training might take a few minutes.

In [72]:
from surprise import SVD
from surprise import evaluate, print_perf

# TODO: fill the code for evaluating the model based on SVD
# We'll use the SVD algorithm.
algo = SVD()

# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, data, measures=['rmse', 'mae'])

print_perf(perf)

/home/omar/anaconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/home/omar/anaconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 4.3774
MAE:  3.3125
------------
Fold 2
RMSE: 4.3765
MAE:  3.3142
------------
------------
Mean RMSE: 4.3770
Mean MAE : 3.3133
------------
------------
        Fold 1  Fold 2  Mean    
RMSE    4.3774  4.3765  4.3770  
MAE     3.3125  3.3142  3.3133  


The above code was hopefully short, and it's mainly for showing the power of the library. Now that you have trained and evaluated the recommendation algorithm, let's try to find the predicted rating for a single user and item.

In [73]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(98)  # raw item id (as in the ratings file). They are **strings**!

# TODO get a prediction for user with uid and item iid
pred = algo.predict(uid, iid, r_ui=0, verbose=True)

user: 196        item: 98         r_ui = 0.00   est = -3.25   {'was_impossible': False}


If you are interested in knowing what the joke was for item 98, you can check the dataset. By default, the dataset will be downloaded in your home directory, under `$HOME/.surprise_data/jester/`. The file `jester_items.dat` has the text of the jokes. 😉

Finally, feel free to explore the library further. It might come in handy for your future projects!
